# First Create a Elastic Search Connection

In [11]:
import re
import os
import sys
import json
import time
import json
import yaml
import math
import pytz
import locale
import logging
import tzlocal
import calendar
import datetime
import requests
import dateutil.parser

from os import listdir
from os.path import isfile, join

from datetime import datetime
from datetime import timedelta
from elasticsearch import Elasticsearch as ES, RequestsHttpConnection as RC

es_host="esnodebal"
es_port=9200

host_params="http://"+es_host+":"+str(es_port)
es = ES(hosts=[host_params])
es.info()

{'name': 'esnodebal',
 'cluster_name': 'nyxdata-cluster',
 'cluster_uuid': 'AwCOybaZTfGwoHolOXyFbg',
 'version': {'number': '6.5.4',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': 'd2ef93d',
  'build_date': '2018-12-17T21:17:40.758843Z',
  'build_snapshot': False,
  'lucene_version': '7.5.0',
  'minimum_wire_compatibility_version': '5.6.0',
  'minimum_index_compatibility_version': '5.0.0'},
 'tagline': 'You Know, for Search'}

# The reads all the alert rules from the folder

In [13]:

# PATH TO MY ELASTALERT
elastpath="../../elastalert"

alerts = [f for f in listdir(elastpath) if isfile(join(elastpath, f))]
for alert in alerts:
    with open(elastpath+"/"+alert, 'r') as stream:
        try:
            
            alertobj=yaml.safe_load(stream)
            index=alertobj["index"]
            timestamp_field="@timestamp"
            if "timestamp_field" in alertobj:
                timestamp_field=alertobj["timestamp_field"]
            print("%s => %s => %s" %(alert,index,timestamp_field))
            id="elastspace:index-pattern:"+index.replace("*","star").lower()
    
            k=alertobj
            timeframe="(Timeframe "+" ".join([str(k["timeframe"][key])+" "+key for key in k["timeframe"]])+")"          
    
# USE elastspace space for elast alerts
            newindpat={
              'index-pattern': {
                'title': index,
                'timeFieldName': timestamp_field,
                'fields': '[]'
              },
              'type': 'index-pattern',
              'namespace': 'elastspace'
            }
            res=es.index(index=".kibana",id=id,body=newindpat,doc_type="doc")
            visid="elastspace:visualization:"+alert.replace(".","dot").lower()

# CREATE TIMELION VISUALIZATION

            timelion=".es(index='elastalert',q='rule_name:"+alertobj["name"]+"').yaxis(2).bars().label('Alerts sent'),"
            timelion+=".es(index=\'"+index+"\',timefield=\'"+timestamp_field+"\'"
        
            if "filter" in alertobj and "query_string" in alertobj["filter"][0] and "query" in alertobj["filter"][0]["query_string"]:
                timelion+=",q=\'"+alertobj["filter"][0]["query_string"]["query"]+"\'"
            
        
            timelion+=")"
            
            if "threshold" in alertobj:
                timelion+=",.value("+str(alertobj["threshold"])+").label(\'Threshold "+timeframe+"\')"

            if "num_events" in alertobj:
                timelion+=",.value("+str(alertobj["num_events"])+").label(\'Num. Events "+timeframe+"\')"
                
            newvis={
              'visualization': {
                'title': alertobj["name"]+" ("+alertobj["type"]+")",
                'visState': '{"type":"timelion","title":\"'+alertobj["name"]+'\","params":{"expression":\"'+timelion+'\","interval":"auto"}}',
                'uiStateJSON': '{}',
                'description': '',
                'version': 1,
                'kibanaSavedObjectMeta': {
                  'searchSourceJSON': '{}'
                }
              },
              'type': 'visualization',
              'namespace': 'elastspace'
            }
# ADDING THE VISUALIZATION TO THE SPACE            
            res=es.index(index=".kibana",id=visid,body=newvis,doc_type="doc")
            
        except yaml.YAMLError as exc:
            print(exc)


CheckOptiboard.yaml => pulsestatus* => generationtime
CheckDisksInfra.yaml => nyx_infra_disk* => @timestamp
CheckLogin.yml => nyx_apicalls* => @timestamp
CheckAMQ.yml => nyx_module_info* => @timestamp
CheckDisksOptiboards.yaml => pulsestatus* => generationtime
CheckModules.yaml => nyx_module_info* => @timestamp
CheckMaximo.yaml => nyx_log* => @timestamp
CheckLogstash.yaml => logstash-* => @timestamp
